<a href="https://colab.research.google.com/github/Epilef86/DNC/blob/main/Spark_UI_Pratica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1º Passo: instalar pyspark

In [25]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


2º Passo: para acessar o sparkui através do colba, é necessário instalar o ngrok, que é uma ferramenta que possibilita que acesse através de um servidor que está no google colab. 

In [26]:
!wget -qnc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n -q ngrok-stable-linux-amd64.zip 

Quando der um refresh os arquivos vão aparecer ao lado, ou seja, o ngrok já está na instalado na máquina

In [27]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .config('spark.ui.port', '4050')
    .appName("SparkUI Introdução")
    .getOrCreate()
)

A única configuração que vai ser colocado a mais é que a porta do sparkui estará na 4050, enquanto padrão (default) é 4040. Faz essa mudança por orientação ao ngrok pq ele mesmo usa essa porta.

In [28]:
!./ngrok authtoken 2Lh7Zu0kns54bTSQCTrDDYkyvNH_71s7E3phX7wCyU4nuAFpH
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
https://7a01-34-125-229-124.ngrok.io


Explicando código acima: na primeira linha, eu to falando pro ngrok que o código de autenticação é esse dai. Pra pegar o código de autenticação precisa ser feito cadastro gratuitamente e confirmar o email. Vai na opção de Your Authtoken: copia e cola na primeira linha, igual ao de cima

O !sleep 10 é porque entre as linhas de execução 2 e 4, é necessário que se espere 10 segundos 

A última função, simplesmente pega a URL pública do ngrok que vai ser disponibilizada e já trás pra tela.

In [33]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, TimestampType 

schema_remetente_destinatario = StructType([
    StructField('nome', StringType()),
    StructField('banco', StringType()),
    StructField('tipo', StringType()),
])

schema_base_pix = StructType([
    StructField('id_transacao', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('remetente ', schema_remetente_destinatario),
    StructField('destinatario', schema_remetente_destinatario),
    StructField('transaction_date', TimestampType()),
    StructField('chave_pix', StringType()),
    StructField('fraude', IntegerType())
])

caminho_json = './pix_transactions.json'

df = spark.read.json(
    caminho_json,
    schema = schema_base_pix,
    timestampFormat = "yyyy-MM-dd"
)


Lemos o json, será que já apareceu no sparkui? Não porque ainda não chamei nenhuma ação por este motivo o código não executou. 

Se eu chamar por exemplo select

In [34]:
df.select('id_transacao')

DataFrame[id_transacao: int]

Eu não estou chamando nenhuma ação, apenas selecionando.Digamos que eu queria mostrar os dados na tela com .show()

In [35]:
df.show()

+------------+--------+----------+--------------------+-------------------+---------+------+
|id_transacao|   valor|remetente |        destinatario|   transaction_date|chave_pix|fraude|
+------------+--------+----------+--------------------+-------------------+---------+------+
|        1000|    7.05|      null|{Gabriel Cunha, I...|2022-03-19 00:00:00|      cpf|     0|
|        1001|   37.28|      null|{Diego Souza, XP,...|2021-01-26 00:00:00|aleatoria|     0|
|        1002|  282.73|      null|{Nicole Nunes, BT...|2022-05-31 00:00:00|aleatoria|     0|
|        1003| 8447.92|      null|{Maria Fernanda C...|2022-07-04 00:00:00|aleatoria|     0|
|        1004|   58.51|      null|{Isabel Silva, C6...|2021-09-11 00:00:00|aleatoria|     0|
|        1005| 6655.12|      null|{Anthony Carvalho...|2022-02-11 00:00:00|  celular|     0|
|        1006| 9912.25|      null|{Eloah Monteiro, ...|2022-05-10 00:00:00|      cpf|     0|
|        1007| 8212.91|      null|{Sophie Rocha, BT...|2022-08-28 00:0

O .show() já é uma ação, então já deve estar aparecendo no spark. Lá mostra a duração do processamento de 4s, posso entrar dentro desse job na aba descriptions, clicando no link. Lá vemos que o inpush dos dados foi 25.2 MB, oq bate com nosso arquivo, então ele leu e mostrou as 20 primeiras linhas. 

Event time podemos ver que o dado foi instanciado a pouco mais de 10 minutos, mas a execução do código foi mais ou menos 4s

DAG Visualization primeiro escaneou o json, realizou peueno processamento e mapeou as partes internamente para mostrar pra gente.

Se eu clicar em Description na parte de stage, eu vou pra dentro do stage, leu 25.2MB, não ghouve nenhum erro, durou 3s

Se quiser fazer algo mais complexo, na coluna de remetente quando printado o df.show() ele tem outra estrutura. Então por exemplo, se eu fizer o select do remetente e o quero saber o nome do remetente

In [43]:
df.select("destinatario.nome").show()

+--------------------+
|                nome|
+--------------------+
|       Gabriel Cunha|
|         Diego Souza|
|        Nicole Nunes|
|Maria Fernanda Ca...|
|        Isabel Silva|
|    Anthony Carvalho|
|      Eloah Monteiro|
|        Sophie Rocha|
|      Pietro Ribeiro|
|      Eloah Teixeira|
|     Emanuella Sales|
|    Valentina Campos|
|       Stella Araujo|
|     Benicio Costela|
|      Joao Fernandes|
|   Gabriela da Rocha|
|      Larissa Aragao|
|           Theo Dias|
|        Danilo Jesus|
|       Bruno Correia|
+--------------------+
only showing top 20 rows



Foi executado uma vez o .show(), vamos ver como aparace lá no sparkui: clica em jobs e observe que já demorou menos tempo para executor, nesse caso 2s. Isso pq estamos consultando menos dados, então o spark vai ser muito mais rápido ao processar. A DAG vai ser a mesma, o input também. 
Clicando no descriptions, indo na aba SQL/DataFRame, se observar a ultima execução vai ser mostrada algumas metricas interessantes: escaneou 25 MB e o output foi de 21 linhas, foi lido um arquivo e o tempo para ler o metadado. Quanto mais partições, mais demora pra ler o metadado

Agora vamos ver como o spark se comporta quando escrevemos essas dados em uma area especifica. Escrevendo no formato parquet

In [46]:
df.write.mode('overwrite').partitionBy('chave_pix').parquet('outpute/pix')

Olhando em arquivos ao lado apareceu outpute e dentro dela tem a base pix particionada pelas chave que tem nela. Agora vamos ver como sparkui demonstra isso

Temos o último job que demorou 5s. Observe na Description que o método é o parquet, antes era showString. 

A DAG mostra como paraleliza os dados: escanou json, projetou, mapeou e fez um sorted para particionar na escrita. No SQL/DataFrame vemos que o numero de linhas foi 100 foi de 1 arquivo de tamanho 25 MB. O que foi escrito foi pra 1000 kB, então o parquet tem o poder de pegar os dados e deixar muito menor, escreveu 4 arquivos 